In [ ]:
def sliding_window(img, h, w, x_step, y_step):
    """
    Permet d'appliquer une fenetre glissante sur une image. L'algorithme se contente de renvoyer
    les parties de l'image correspondante à une unique passe d'une fenetre unique sans appliquer de classification.
    :param img: Image sur laquelle appliquer la fenetre glissante.
    :param h: Hauteur de la fenetre.
    :param w: Largeur de la fenetre.
    :param x_step: Pas de la fenetre en hauteur.
    :param y_step: Pas de la fenetre en largeur.
    :return: Liste des parties de l'image correspondant à la fenetre glissante. Et les coordonnées de ces parties.
    """
    img_parts = []
    parts_coords = []
    for i in range(0, img.shape[0] - h + 1, x_step):
        for j in range(0, img.shape[1] - w + 1, y_step):
            upper_left_corner = (i, j)
            lower_right_corner = (i + h, j + w)
            img_parts.append(
                img[
                    upper_left_corner[0] : lower_right_corner[0],
                    upper_left_corner[1] : lower_right_corner[1],
                ]
            )
            parts_coords.append((upper_left_corner, lower_right_corner))
    return img_parts, parts_coords

In [ ]:
def get_IOU(window1, window2):
    """
    Cet algorithme permet de calculer l'Intersection over Union (IoU), soit aire de recouvrement, entre deux fenetres.
    :param window1: Fenetre 1. Format ((upper_left_x, upper_left_y), (lower_right_x, lower_right_y)).
    :param window2: Fenetre 2.
    :return float: Aire de recouvrement entre les deux fenetres.
    """
    upper_left_1, lower_right_1 = window1
    upper_left_2, lower_right_2 = window2

    upper_left_inter = (
        max(upper_left_1[0], upper_left_2[0]),
        max(upper_left_1[1], upper_left_2[1]),
    )
    lower_right_inter = (
        min(lower_right_1[0], lower_right_2[0]),
        min(lower_right_1[1], lower_right_2[1]),
    )

    upper_left_union = (
        min(upper_left_1[0], upper_left_2[0]),
        min(upper_left_1[1], upper_left_2[1]),
    )

    lower_right_union = (
        max(lower_right_1[0], lower_right_2[0]),
        max(lower_right_1[1], lower_right_2[1]),
    )

    inter_width = max(0, lower_right_inter[0] - upper_left_inter[0])
    inter_height = max(0, lower_right_inter[1] - upper_left_inter[1])
    union_width = max(0, lower_right_union[0] - upper_left_union[0])
    union_height = max(0, lower_right_union[1] - upper_left_union[1])

    inter_area = inter_height * inter_width
    union_area = union_height * union_width

    IOU = inter_area / union_area if union_area > 0 else 0

In [ ]:
def group_windows_by_IOU(windows_list, decision_criteria=0.5):
    """
    Cet algorithme permet de grouper les fenetres qui se recouvrent en fonction de l'Intersection over Union (IoU).
    :param windows_list: Liste de fenetres. Format ((upper_left_x, upper_left_y), (lower_right_x, lower_right_y), score).
    :param decision_criteria: Seuil de recouvrement pour regrouper les fenetres.
    :return: Liste de fenetres regroupées.
    """
    grouped_windows = []
    grouped_scores = []
    while windows_list:  # Tant qu'il reste des fenetres à traiter
        current_window = windows_list.pop(0)
        windows_list_temp = [current_window]  # On commence avec la fenetre courante
        for compared_window in windows_list_temp:
            IOU = get_IOU(current_window[:2], compared_window[:2])
            if IOU > decision_criteria:
                # Si les fenetres dépassent le seuil de recouvrement, on les regroupe
                windows_list_temp.append(compared_window)
                windows_list_temp.remove(compared_window)
        if len(windows_list_temp) > 0:
            grouped_windows.append(windows_list_temp)


def non_maxima_suppression(windows_list, decision_criteria=0.5):
    """
    Cet algorithme permet de supprimer les fenetres qui se recouvrent trop, en gardant la fenetre avec le score le plus haut.
    :param windows_list: Liste de fenetres. Format ((upper_left_x, upper_left_y), (lower_right_x, lower_right_y), score).
    :param score_list: Liste des scores associés aux fenetres.
    :param decision_criteria: Seuil de recouvrement pour regrouper les fenetres.
    """
    grouped_windows = group_windows_by_IOU(windows_list, decision_criteria)
    best_windows = []
    for group in grouped_windows:
        if len(group) == 0:
            continue
        elif len(group) == 1:
            best_window = group[0]
        else:
            best_window = max(
                group, key=lambda x: x[2]
            )  # On prend la fenetre avec le score le plus haut
        best_windows.append(best_window)
    return best_windows

In [ ]:
def detect_ecocup(img, classifier):
    """


    Cette fonction permet de détecter les gobelets en plastique dans une image.


    :param img: Image sur laquelle appliquer la détection.


    :param classifier: Classificateur entrainé à utiliser pour la détection.


    :return: Liste des coordonnées des gobelets détectés.


    """

    sizes = range(20, 200, 20)

    squares = [(s, s) for s in sizes]

    rectangles = [(h, w) for h in sizes for w in sizes if h != w]

    hw_list = squares + rectangles

    step_squares = [(s, s) for s in sizes]
    step_rectangles = [(h, w) for h in sizes for w in sizes if h != w]
    step_list = step_squares + step_rectangles

    score_window_list = []

    for h, w in hw_list:
        for x_step, y_step in step_list:
            img_parts, windows_list = sliding_window(img, h, w, x_step, y_step)
            score_list = []
            for img_part in img_parts:
                if classifier.predict(img_part) == 1:
                    score_list.append(classifier.score(img_part))
                else:
                    score_list.append(0)
            # grouper score et fenetre
            score_window_list_temp = [
                (window, score) for window, score in zip(windows_list, score_list)
            ]
            # Filtrer les 0
            score_window_list_temp = [
                (window, score) for window, score in score_window_list_temp if score > 0
            ]
            score_window_list.append(score_window_list_temp)

    # Ne garder que les meilleures fenetres
    best_windows = non_maxima_suppression(score_window_list)

    # Extraire les coordonnées des meilleures fenetres
    best_coords = [(window[0], window[1]) for window in best_windows]

    return best_coords